In [1]:
import warnings

warnings.filterwarnings('ignore')
import numpy as _np
#from bgen_reader import open_bgen
#from os.path import join
#from pandas_plink import read_plink1_bin
#from pandas_plink import get_data_folder
# import matplotlib.pyplot as plt
#from bgen_reader import open_bgen
#from bed_reader import open_bed, sample_file
# import pandas as pd
# import os
# from pysnptools.distreader import Bgen, DistMemMap
# from pathlib import Path
from bed_reader import open_bed as _open_bed
# from pysnptools.snpreader import SnpData, SnpMemMap
# from pysnptools.util import log_in_place
# import matplotlib.pyplot as plt
# from scipy.stats import gaussian_kde
from KDEpy import FFTKDE as _FFTKDE
import pandas as _pd
# from pybgen import PyBGEN
import multiprocess as _mp

In [65]:
def MI_continuous_SNP(a,
                      b,
                      a_min,
                      a_max,
                      N=500,
                      kernel="epa",
                      bw="silverman",
                      machine_err=1e-24):
    """
    calculate mutual information between continuous outcome and an SNP variable of 0,1,2
    assume no missing data
    """
    # first estimate the pmf
    p0 = _np.sum(b == 0) / len(b)
    p1 = _np.sum(b == 1) / len(b)
    p2 = 1. - p0 - p1
    # estimate cond density
    _b0 = (b == 0)
    if _np.sum(
            _b0
    ) > 2:  # here proceed to kde only if there are more than 5 data points
        y_cond_p0 = _FFTKDE(kernel=kernel, bw=bw).fit(data=a[_b0])
#         y_cond_p0 = gaussian_kde(a[_b0])
    else:
        y_cond_p0 = _np.zeros_like
    _b1 = (b == 1)
    if _np.sum(_b1) > 2:
        y_cond_p1 = _FFTKDE(kernel=kernel, bw=bw).fit(data=a[_b1])
#         y_cond_p1 = gaussian_kde(a[_b1]) # this thing uses Scott's rule instead of Silverman defaulted by FFTKDE and R density
    else:
        y_cond_p1 = _np.zeros_like
    _b2 = (b == 2)
    if _np.sum(_b2) > 2:
        y_cond_p2 = _FFTKDE(kernel=kernel, bw=bw).fit(data=a[_b2])


#         y_cond_p2 = gaussian_kde(a[_b2])
    else:
        y_cond_p2 = _np.zeros_like
    joint = _np.empty((N, 3))
    a_temp = _np.linspace(a_min, a_max, num=N)
    joint[:, 0] = y_cond_p0(a_temp) * p0
    joint[:, 1] = y_cond_p1(a_temp) * p1
    joint[:, 2] = y_cond_p2(a_temp) * p2
    joint[
        joint <
        machine_err] = machine_err  # set a threshold to avoid numerical errors
    forward_euler_step = a_temp[1] - a_temp[0]
    #     print("total measure:", _np.sum(joint)*forward_euler_step)
    temp_log = _np.log(joint)
    #     temp_log = _np.nan_to_num(temp_log, nan = 0)
    temp1 = _np.log(_np.sum(joint, 1))
    #     temp1 = _np.nan_to_num(temp1, nan = 0)
    temp_log = temp_log - temp1.reshape(-1, 1)
    temp2 = _np.log(_np.sum(joint, 0)) + _np.log(forward_euler_step)
    #     temp2 = _np.nan_to_num(temp2, nan = 0)
    temp_log = temp_log - temp2.reshape(1, -1)
    # print(fhat_mat * temp_log)
    temp_mat = joint * temp_log
    #     temp_mat = _np.nan_to_num(temp_mat, nan=0.) # numerical fix
    mi_temp = _np.sum(temp_mat) * forward_euler_step
    return mi_temp


def MI_binary_SNP(a, b):
    """
    calculate mutual information between binary outcome and an SNP variable of 0,1,2
    assume no missing data
    """
    # first estimate the pmf of SNP
    p0 = _np.sum(b == 0) / len(b)
    p1 = _np.sum(b == 1) / len(b)
    p2 = 1. - p0 - p1
    b_marginal = _np.array([p0, p1, p2])
    # estimate pmf of the binary outcome
    a_p0 = _np.sum(a == 0) / len(a)
    a_p1 = _np.sum(a == 1) / len(a)
    a_marginal = _np.array([a_p0, a_p1]).reshape(-1, 1)
    # estimate the cond density
    joint = _np.zeros((2, 3))
    _b0 = (b == 0)
    joint[0, 0] = _np.sum(a[_b0] == 0) / len(a)
    joint[1, 0] = _np.sum(a[_b0] == 1) / len(a)
    _b1 = (b == 1)
    joint[0, 1] = _np.sum(a[_b1] == 0) / len(a)
    joint[1, 1] = _np.sum(a[_b1] == 1) / len(a)
    _b2 = (b == 2)
    joint[0, 2] = _np.sum(a[_b2] == 0) / len(a)
    joint[1, 2] = _np.sum(a[_b2] == 1) / len(a)

    _ = a_marginal * b_marginal
    _ = joint / _
    __ = joint * _np.log(_)
    __ = _np.nan_to_num(__, nan=0.0)  # for possible nuemrical issues

    mi_temp = _np.sum(__)

    return mi_temp


# make this function available
def MI_bivariate_continuous(a,
                            b,
                            a_N=300,
                            b_N=300,
                            kernel="epa",
                            bw="silverman",
                            norm=2,
                            machine_err=1e-24):
    """
    (Single Core version) calculate mutual information on bivariate continuous r.v..
    """
    _ = np.argsort(a)
    data = _np.hstack((a[_].reshape(-1, 1), b[_].reshape(-1, 1)))
    grid, joint = _FFTKDE(kernel=kernel, norm=norm).fit(data).evaluate(
        (a_N, b_N))
    joint = joint.reshape(b_N, -1).T
    # this gives joint as a (a_N, b_N) array, following example: https://kdepy.readthedocs.io/en/latest/examples.html#the-effect-of-norms-in-2d
    a_forward_euler_step = grid[b_N, 0] - grid[0, 0]
    b_forward_euler_step = grid[1, 1] - grid[0, 1]
    joint[joint < machine_err] = machine_err
    log_a_marginal = _np.log(_np.sum(joint, 1)) + _np.log(b_forward_euler_step)
    log_b_marginal = _np.log(_np.sum(joint, 0)) + _np.log(a_forward_euler_step)
    log_joint = _np.log(joint)
    mi_temp = _np.sum(
        joint *
        (log_joint - log_a_marginal.reshape(-1, 1) - log_b_marginal.reshape(
            1, -1))) * a_forward_euler_step * b_forward_euler_step
    return mi_temp


# make this function available
def MI_binary_continuous(a,
                         b,
                         b_min,
                         b_max,
                         N=500,
                         kernel="epa",
                         bw="silverman",
                         machine_err=1e-24):
    return MI_continuous_SNP(a=b,
                             b=a,
                             a_min=b_min,
                             a_max=b_max,
                             N=N,
                             kernel=kernel,
                             bw=bw,
                             machine_err=machine_err)


# outcome_iid should be a  list of strings for identifiers
def continuous_filter_plink(bed_file,
                            bim_file,
                            fam_file,
                            outcome,
                            outcome_iid,
                            a_min=1.2345654321,
                            a_max=2.34565432,
                            N=500,
                            kernel="epa",
                            bw="silverman",
                            machine_err=1e-24):
    """
    (Single Core version) take plink files to calculate the mutual information between the continuous outcome and many SNP variables.
    """
    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    if a_min == 1.2345654321:
        a_min = _np.min(outcome) - _np.std(outcome)
    if a_max == 2.34565432:
        a_max = _np.max(outcome) + _np.std(outcome)
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]
    MI_UKBB = _np.zeros(len(bed1_sid))
    for j in range(len(MI_UKBB)):
        _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
        _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
        _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
        _SNP = _SNP[_SNP != -127]  # remove missing SNP
        MI_UKBB[j] = MI_continuous_SNP(a=_outcome,
                                       b=_SNP,
                                       a_min=a_min,
                                       a_max=a_max,
                                       N=N,
                                       kernel=kernel,
                                       bw=bw,
                                       machine_err=machine_err)
    return MI_UKBB


def binary_filter_plink(bed_file, bim_file, fam_file, outcome, outcome_iid):
    """
    (Single Core version) take plink files to calculate the mutual information between the binary outcome and many SNP variables.
    """
    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]
    MI_UKBB = _np.zeros(len(bed1_sid))
    print(outcome)
    for j in range(len(MI_UKBB)):
        _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
        _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
        _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
        _SNP = _SNP[_SNP != -127]  # remove missing SNP
        MI_UKBB[j] = MI_binary_SNP(a=_outcome, b=_SNP)
    return MI_UKBB


def continuous_filter_plink_parallel(bed_file,
                                     bim_file,
                                     fam_file,
                                     outcome,
                                     outcome_iid,
                                     a_min=1.2345654321,
                                     a_max=2.34565432,
                                     N=500,
                                     core_num="NOT DECLARED",
                                     multp=1,
                                     kernel="epa",
                                     bw="silverman",
                                     machine_err=1e-24):
    """
    (Multiprocessing version) take plink files to calculate the mutual information between the continuous outcome and many SNP variables.
    """
    if core_num == "NOT DECLARED":
        core_num = _mp.cpu_count()
    else:
        assert core_num <= _mp.cpu_count(
        ), "Declared number of cores used for multiprocessing should not exceed number of cores on this machine."
    assert core_num >= 2, "Multiprocessing should not be used on single-core machines."

    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    if a_min == 1.2345654321:
        a_min = _np.min(outcome) - _np.std(outcome)
    if a_max == 2.34565432:
        a_max = _np.max(outcome) + _np.std(outcome)
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]

    def _continuous_filter_plink_slice(_slice):
        _MI_slice = _np.zeros(len(_slice))
        k = 0
        for j in _slice:
            _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
            _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
            _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
            _SNP = _SNP[_SNP != -127]  # remove missing SNP
            _MI_slice[k] = MI_continuous_SNP(a=_outcome,
                                             b=_SNP,
                                             a_min=a_min,
                                             a_max=a_max,
                                             N=N,
                                             kernel=kernel,
                                             bw=bw,
                                             machine_err=machine_err)
            k += 1
        return _MI_slice

    # multiprocessing starts here
    ind = _np.arange(len(bed1_sid))
    with _mp.Pool(core_num) as pl:
        MI_UKBB = pl.map(_continuous_filter_plink_slice,
                         _np.array_split(ind, core_num * multp))
    MI_UKBB = _np.hstack(MI_UKBB)
    return MI_UKBB


def binary_filter_plink_parallel(bed_file,
                                 bim_file,
                                 fam_file,
                                 outcome,
                                 outcome_iid,
                                 core_num="NOT DECLARED",
                                 multp=1):
    """
    (Multiprocessing version) take plink files to calculate the mutual information between the binary outcome and many SNP variables.
    """
    if core_num == "NOT DECLARED":
        core_num = _mp.cpu_count()
    else:
        assert core_num <= _mp.cpu_count(
        ), "Declared number of cores used for multiprocessing should not exceed number of cores on this machine."
    assert core_num >= 2, "Multiprocessing should not be used on single-core machines."

    bed1 = _open_bed(filepath=bed_file,
                     fam_filepath=fam_file,
                     bim_filepath=bim_file)
    gene_iid = _np.array(list(bed1.iid))
    bed1_sid = _np.array(list(bed1.sid))
    outcome = outcome[_np.intersect1d(outcome_iid,
                                      gene_iid,
                                      assume_unique=True,
                                      return_indices=True)[1]]
    # get genetic indices
    gene_ind = _np.intersect1d(gene_iid,
                               outcome_iid,
                               assume_unique=True,
                               return_indices=True)[1]

    def _binary_filter_plink_slice(_slice):
        _MI_slice = _np.zeros(len(_slice))
        k = 0
        for j in _slice:
            _SNP = bed1.read(_np.s_[:, j], dtype=_np.int8).flatten()
            _SNP = _SNP[gene_ind]  # get gene iid also in outcome iid
            _outcome = outcome[_SNP != -127]  # remove missing SNP in outcome
            _SNP = _SNP[_SNP != -127]  # remove missing SNP
            _MI_slice[k] = MI_binary_SNP(a=_outcome, b=_SNP)
            k += 1
        return _MI_slice

    # multiprocessing starts here
    ind = _np.arange(len(bed1_sid))
    with _mp.Pool(core_num) as pl:
        MI_UKBB = pl.map(_binary_filter_plink_slice,
                         _np.array_split(ind, core_num * multp))
    MI_UKBB = _np.hstack(MI_UKBB)
    return MI_UKBB


def binary_filter_csv(csv_file,
                      csv_names=[],
                      N=500,
                      kernel="epa",
                      bw="silverman",
                      machine_err=1e-24):
    # outcome is the first variable by default; if other specifications are needed, put it the first item in csv_names
    if _np.array(csv_names).size == 0:
        print(
            "Variable names not provided -- start reading variable names from csv file now, might take some time, depending on the csv file size."
        )
        csv_names = pd.read_csv(csv_file, index_col=0,
                                nrows=0).columns.tolist()
        print("Reading variable names from csv file finished.")
    else:
        csv_names = _np.array(csv_names)
    MI_csv = np.empty(len(csv_names) - 1)
    for j in np.arange(len(csv_names) - 1):
        __ = csv_names[0, j + 1]
        _ = pd.read_csv(outcome_file,
                        skipinitialspace=True,
                        usecols=__,
                        encoding='unicode_escape').dropna()
        MI_csv[j] = MI_binary_continuous(
            a=_[:, 0],
            b=_[:, 1],
            b_min=np.min(_[:, 1]) - np.std(_[:, 1]),
            b_max=np.min(_[:, 1]) + np.std(_[:, 1]),
            N=N,
            kernel=kernel,
            bw=bw,
            machine_err=machine_err)
    return MI_csv


def continuous_filter_csv(csv_file,
                          csv_names=[],
                          a_N=300,
                          b_N=300,
                          kernel="epa",
                          bw="silverman",
                          norm=2,
                          machine_err=1e-24):
    # outcome is the first variable by default; if other specifications are needed, put it the first item in csv_names
    if _np.array(csv_names).size == 0:
        print(
            "Variable names not provided -- start reading variable names from csv file now, might take some time, depending on the csv file size."
        )
        csv_names = pd.read_csv(csv_file, index_col=0,
                                nrows=0).columns.tolist()
        print("Reading variable names from csv file finished.")
    else:
        csv_names = _np.array(csv_names)
    MI_csv = np.empty(len(csv_names) - 1)
    for j in np.arange(len(csv_names) - 1):
        __ = csv_names[0, j + 1]
        _ = pd.read_csv(outcome_file,
                        skipinitialspace=True,
                        usecols=__,
                        encoding='unicode_escape').dropna()
        MI_csv[j] = MI_bivariate_continuous(a=_[:, 0],
                                            b=_[:, 1],
                                            a_N=a_N,
                                            b_N=b_N,
                                            kernel=kernel,
                                            bw=bw,
                                            norm=norm,
                                            machine_err=machine_err)
    return MI_csv


def binary_filter_csv_parallel():
    pass
    return


def continuous_filter_csv_parallel():
    pass
    return

In [66]:
import numpy as np

a = np.random.binomial(2, .3, 200)
b = np.random.binomial(2, .3, 200)
print(MI_binary_SNP(a, b))

a = np.random.normal(size=2000)
b = np.random.normal(size=2000)

print(MI_bivariate_continuous(a, b))
a_MI = np.zeros(500)
b_MI = np.zeros(500)
for i in np.arange(500):
    np.random.seed(i)
    a = np.random.normal(size=2000)
    b = np.random.normal(size=2000)
    a_MI[i] = MI_bivariate_continuous(a, a)
    b_MI[i] = MI_bivariate_continuous(a, b)
print(np.mean(a_MI), np.mean(b_MI))
print(a_MI - b_MI)

0.004420667306593115
0.0010108923378704799
0.2057681739689726 0.0010578902306832203
[0.19665037 0.20765675 0.20149238 0.20915676 0.19420115 0.20309203
 0.19189506 0.20092077 0.20624126 0.21524621 0.19251016 0.20899591
 0.20482374 0.20010838 0.21296618 0.20566429 0.21008501 0.21164677
 0.21190437 0.21398222 0.20474392 0.2025692  0.2059527  0.20338707
 0.2024347  0.21399576 0.20512102 0.20041197 0.20620582 0.20499534
 0.19634174 0.20248316 0.20182316 0.19884378 0.20128408 0.20911378
 0.21186764 0.19951603 0.20851031 0.20950583 0.20303601 0.21459109
 0.20094591 0.20061785 0.20846891 0.20568283 0.21534267 0.19937844
 0.21296178 0.20547831 0.20801865 0.2054691  0.20077725 0.20222824
 0.20739636 0.20894973 0.19490489 0.20784496 0.20147222 0.19867544
 0.20819772 0.20573874 0.20588939 0.2039781  0.21042278 0.20692482
 0.20606303 0.20662075 0.20799746 0.20689041 0.20625125 0.2007687
 0.21707061 0.19756141 0.20310103 0.19559443 0.20321105 0.2037693
 0.21179311 0.1987594  0.19965353 0.20415345 0.

In [4]:
import numpy as np
from bed_reader import open_bed

bed_file = r"./fastHDMI/tests/sim/sim1.bed"
bim_file = r"./fastHDMI/tests/sim/sim1.bim"
fam_file = r"./fastHDMI/tests/sim/sim1.fam"

_bed = open_bed(filepath=bed_file,
                fam_filepath=fam_file,
                bim_filepath=bim_file)
outcome = np.random.rand(_bed.iid_count)
outcome_iid = _bed.iid

true_beta = np.array([4.2, -2.5, 2.6])
for j in np.arange(3):
    outcome += true_beta[j] * _bed.read(_np.s_[:, j], dtype=_np.int8).flatten()
    print(_bed.read(_np.s_[:, j], dtype=_np.float64).flatten())

iid_ind = np.random.permutation(np.arange(_bed.iid_count))
outcome = outcome[iid_ind]
outcome_iid = outcome_iid[iid_ind]

MI_continuous = continuous_filter_plink_parallel(bed_file=bed_file,
                                                 bim_file=bim_file,
                                                 fam_file=fam_file,
                                                 a_min=np.min(outcome) - 10,
                                                 a_max=np.max(outcome) + 10,
                                                 outcome=outcome,
                                                 outcome_iid=outcome_iid,
                                                 multp=2)

MI_continuous[:20]

[1. 0. 1. ... 2. 1. 1.]
[1. 2. 0. ... 0. 2. 1.]
[2. 1. 2. ... 2. 2. 2.]


array([0.47819885, 0.12476599, 0.13383907, 0.00092895, 0.00205855,
       0.00120627, 0.00093233, 0.0016023 , 0.00124484, 0.00118684,
       0.00220929, 0.00129694, 0.00157462, 0.00157844, 0.00161315,
       0.00191134, 0.00128113, 0.00191086, 0.00172114, 0.00105449])

In [5]:
bed_file = r"./fastHDMI/tests/sim/sim1.bed"
bim_file = r"./fastHDMI/tests/sim/sim1.bim"
fam_file = r"./fastHDMI/tests/sim/sim1.fam"

_bed = open_bed(filepath=bed_file,
                fam_filepath=fam_file,
                bim_filepath=bim_file)
outcome = np.random.rand(_bed.iid_count)
outcome_iid = _bed.iid

true_beta = np.array([4.2, -2.5, 2.6])
for j in np.arange(3):
    outcome += true_beta[j] * _bed.read(_np.s_[:, j], dtype=_np.int8).flatten()
    print(_bed.read(_np.s_[:, j], dtype=_np.float64).flatten())

outcome = np.random.binomial(1, np.tanh(outcome / 2) / 2 + .5)

iid_ind = np.random.permutation(np.arange(_bed.iid_count))
outcome = outcome[iid_ind]
outcome_iid = outcome_iid[iid_ind]

MI_binary = binary_filter_plink_parallel(bed_file=bed_file,
                                         bim_file=bim_file,
                                         fam_file=fam_file,
                                         outcome=outcome,
                                         outcome_iid=outcome_iid)

MI_binary[:20]

[1. 0. 1. ... 2. 1. 1.]
[1. 2. 0. ... 0. 2. 1.]
[2. 1. 2. ... 2. 2. 2.]


array([1.13451556e-01, 3.71182060e-02, 4.63782991e-02, 3.75633377e-05,
       1.46876722e-04, 5.93408692e-04, 8.30733674e-04, 2.16295621e-04,
       5.68758566e-04, 6.74265450e-04, 8.36141781e-05, 7.87215709e-06,
       4.68064596e-04, 4.64006530e-04, 6.09939449e-05, 2.17268302e-05,
       7.85024505e-05, 2.29951500e-05, 6.19267441e-04, 8.78445880e-04])